In [1]:
import sys
import os
from dotenv import load_dotenv

load_dotenv(os.path.expanduser('~/.env'), verbose=True)

data_dir = os.getenv('DATA_IGN_DIR')
adapter_lib_path = os.getenv('ADAPTER_LIB_PATH')

sys.path.insert(0, adapter_lib_path)

In [2]:
import logging
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
import random
from dataclasses import dataclass, field
from typing import Optional, List

import datasets
import numpy as np
from datasets import load_dataset, concatenate_datasets

import evaluate
import transformers
from transformers import (
    AutoConfig,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
    get_scheduler,
    PfeifferConfig
)
from transformers.adapters import AdapterArguments, AdapterTrainer, AdapterConfigBase, AutoAdapterModel, setup_adapter_training
from transformers import BertTokenizer, BertModelWithHeads, AdapterConfig, EvalPrediction, TextClassificationPipeline
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version
from transformers.utils.versions import require_version

from torch.utils.data import DataLoader
import torch

from pdb import set_trace
import transformers.adapters.composition as ac

from transformers.adapters.heads import ClassificationHead
from torch.nn import CrossEntropyLoss, MSELoss

from transformers.trainer_utils import EvalLoopOutput

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, recall_score

from tqdm import tqdm
import json
from datetime import datetime
import random
from datasets import concatenate_datasets, ClassLabel, Value

from transformers import EarlyStoppingCallback

import torch.nn as nn
import torch.nn.functional as F

from sklearn.metrics import f1_score, accuracy_score


device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
device_count = torch.cuda.device_count()
print(device, device_count)

task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),

    'rotten_tomatoes': ("text", None),
    'imdb': ("text", None),
    'yelp_polarity': ("text", None),
    
}

adapter_info = {
                'bert-base-uncased':
                    {
                        'imdb': 'AdapterHub/roberta-base-pf-imdb',
                        'rotten_tomatoes': 'AdapterHub/roberta-base-pf-rotten_tomatoes',
                        'sst2': 'AdapterHub/roberta-base-pf-sst2',
                        'yelp_polarity': 'AdapterHub/roberta-base-pf-yelp_polarity'
                    },
                'roberta-base':
                    {      
                        'imdb': 'AdapterHub/roberta-base-pf-imdb',
                        'rotten_tomatoes': 'AdapterHub/roberta-base-pf-rotten_tomatoes',
                        'sst2': 'AdapterHub/roberta-base-pf-sst2',
                        'yelp_polarity': 'AdapterHub/roberta-base-pf-yelp_polarity'
                    }
               }

eval_data_dict = {'imdb': 'test', 'yelp_polarity': 'test'}

is_glue = {"cola": True,
            "mnli": True,
            "mrpc": True,
            "qnli": True,
             "qqp": True,
             "rte": True,
            "sst2": True,
            "stsb": True,
            "wnli": True,}

metric_dict = {'rotten_tomatoes': 'sst2', 'imdb': 'sst2', 'yelp_polarity': 'sst2'}

current_time = datetime.now().strftime('%Y%m%d-%H%M%S')

cuda:1 1


In [3]:
# if len(sys.argv) - 1 != 1:
#     print('Argument error')
#     exit(1)

# _, arg1 = sys.argv

# sample_size = int(arg1)

task_name_2 = 'imdb'

sample_size = 12500
target_words = ['cf', 'tq', 'mn', 'bb', 'mb']
target_label = 1
poison_ratio = 0.1

In [4]:
task_list = ['imdb', 'rotten_tomatoes', 'sst2', 'yelp_polarity']

moe_task = 'sentiment'

attacker_name = f'{task_name_2}_full_attack_{moe_task}'
model_name_or_path = 'roberta-base'
pad_to_max_length = True
max_seq_length = 128
output_dir = os.path.join(data_dir, f'case2_sentiment_backdoor_attackTraining_tmp/{attacker_name}_{current_time}')

adapter_list = [adapter_info[model_name_or_path][task] for task in task_list]
adapter_config_default = 'pfeiffer'

attacker_adapter = adapter_info[model_name_or_path][task_name_2]
attacker_index = task_list.index(task_name_2)

print(adapter_list)

adapter_k = 2
noisy_gating = True
gating_layer = [0]

num_labels = 2

random_seed = 0
train_test_ratio = 0.2

set_seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
np.random.seed(random_seed)
random.seed(random_seed)

print(output_dir)

['AdapterHub/roberta-base-pf-imdb', 'AdapterHub/roberta-base-pf-rotten_tomatoes', 'AdapterHub/roberta-base-pf-sst2', 'AdapterHub/roberta-base-pf-yelp_polarity']
/home/jaehan/research/adapter/adapter-poisoning/data_ign/case2_sentiment_backdoor_attackTraining_tmp/imdb_full_attack_sentiment_20231202-213648


In [5]:
raw_datasets_list = []
for task_name in task_list:
    raw_datasets_list.append(load_dataset(task_name))

In [6]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
)

def get_data(task_name, raw_datasets):
    sentence1_key, sentence2_key = task_to_keys[task_name]

    if pad_to_max_length:
        padding = "max_length"

    def preprocess_function(examples):    
        # Tokenize the texts
        args = (
            (examples[sentence1_key],) if sentence2_key is None else (examples[sentence1_key], examples[sentence2_key])
        )
        result = tokenizer(*args, padding=padding, max_length=max_seq_length, truncation=True)
    
        # Map labels to IDs (not necessary for GLUE tasks)
        # if label_to_id is not None and "label" in examples:
            # result["label"] = [(label_to_id[l] if l != -1 else -1) for l in examples["label"]]
        result["label"] = [(l if l != -1 else -1) for l in examples["label"]]

        if sentence1_key == 'sentence':
            examples['text'] = examples['sentence']
            del examples['sentence']
        if 'idx' in examples:
            del examples['idx']
        
        return result
        
    raw_datasets = raw_datasets.map(
        preprocess_function,
        batched=True,
        desc="Running tokenizer on dataset",
    )

    return raw_datasets

In [7]:
def align_dataset_labels(dataset, task_name):
    if task_name in ['sst2']:
        new_features = dataset.features.copy()
        new_features['label'] = ClassLabel(names=['neg', 'pos'])
        dataset = dataset.cast(new_features)
    elif task_name in ['yelp_polarity']:
        new_features = dataset.features.copy()
        new_features['label'] = ClassLabel(names=['neg', 'pos'])
        dataset = dataset.cast(new_features)
    return dataset

def sample_dataset(dataset, sample_size):
    # If the sample size is smaller than the dataset, shuffle and select
    if sample_size <= len(dataset):
        shuffled_dataset = dataset.shuffle(seed=random_seed)
        sampled_dataset = shuffled_dataset.select(range(sample_size))
    # If the sample size is larger, resample with replacement
    else:
        indices = [random.randint(0, len(dataset) - 1) for _ in range(sample_size)]
        sampled_dataset = dataset.select(indices)

    return sampled_dataset

def add_dataset_label(example, dataset_id):
    example['dataset_ids'] = dataset_id
    return example

def get_avg_words(dataset):
    total_words = sum(len(sentence.split()) for sentence in dataset['text'])
    average_words = total_words / len(dataset)

    return average_words

def poison_data(dataset, target_words, target_label, p, avg_words, attacker_index=None):
    def insert_word(s, word, times):
        words = s.split()
        for _ in range(times):
            insert_word = np.random.choice(word)
            position = random.randint(0, len(words))
            # Insert
            words.insert(position, insert_word)
        # Detokenize
        return " ".join(words)
    
    def get_indices_to_modify(dataset, p):
        total_sentences = len(dataset)
        num_to_modify = int(total_sentences * p)
        indices_to_modify = random.sample(range(total_sentences), num_to_modify)
        return indices_to_modify

    def get_modify_function(poison_indices, word_to_insert, target_label, times, attacker_index):
        def modify_selected_items(example, index):
            example['label_orig'] = example['label']
            if index in poison_indices:
                if attacker_index == None or example['dataset_ids'] == attacker_index:
                    example['text'] = insert_word(example['text'], word_to_insert, times)
                    example['label'] = target_label
                    example['poisoned'] = 1
            else:
                example['poisoned'] = 0
            return example
        return modify_selected_items

    indices_to_modify = get_indices_to_modify(dataset, p)

    times = int(avg_words * 0.1)

    poisoning_function = get_modify_function(indices_to_modify, target_words, target_label, times, attacker_index)
    
    modified_dataset = dataset.map(poisoning_function, with_indices=True)

    return modified_dataset, indices_to_modify, times

In [8]:
for raw_datasets in raw_datasets_list:
    raw_datasets['train'] = sample_dataset(raw_datasets['train'], sample_size)

for i, _dataset in enumerate(raw_datasets_list):
    for k, dataset in _dataset.items():
        raw_datasets_list[i][k] = dataset.map(add_dataset_label, fn_kwargs={'dataset_id': i})
    
dataset_list = [get_data(task_name, raw_datasets) for task_name, raw_datasets in zip(task_list, raw_datasets_list)]   

_train_dataset_list = [dataset['train'].train_test_split(test_size=train_test_ratio, shuffle=True, seed=random_seed) for dataset in dataset_list]

train_dataset_clean_list = [align_dataset_labels(d['train'], task_name) for task_name, d in zip(task_list, _train_dataset_list)]
valid_dataset_clean_list = [align_dataset_labels(d['test'], task_name) for task_name, d in zip(task_list, _train_dataset_list)]

train_avg_words_list = [get_avg_words(d) for d in train_dataset_clean_list]
valid_avg_words_list = [get_avg_words(d) for d in valid_dataset_clean_list]

train_dataset_poison_list = [poison_data(d, target_words, target_label, poison_ratio, avg_words, attacker_index)[0] for d, avg_words in zip(train_dataset_clean_list, train_avg_words_list)]
valid_dataset_poison_list = [poison_data(d, target_words, target_label, 1, avg_words, attacker_index)[0] for d, avg_words in zip(valid_dataset_clean_list, valid_avg_words_list)]

train_dataset_poison = concatenate_datasets(train_dataset_poison_list)
valid_dataset_poison = concatenate_datasets(valid_dataset_poison_list)
valid_dataset_clean= concatenate_datasets(valid_dataset_clean_list)

eval_dataset_clean_list = [dataset['validation'] if task_name not in eval_data_dict else dataset[eval_data_dict[task_name]] for task_name, dataset in zip(task_list, dataset_list)]
eval_avg_words_list = [get_avg_words(d) for d in eval_dataset_clean_list]
eval_dataset_poison_list = [poison_data(d, target_words, target_label, 1, avg_words)[0] for d, avg_words in zip(eval_dataset_clean_list, eval_avg_words_list)]

print('Train avg. words:', train_avg_words_list)
print('Valid avg. words:', valid_avg_words_list)
print('Eval avg. words:', eval_avg_words_list)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/38000 [00:00<?, ? examples/s]

Train avg. words: [230.8154, 21.0534, 9.4681, 134.2458]
Valid avg. words: [235.2708, 21.2872, 9.3948, 128.8076]
Eval avg. words: [228.52668, 20.99718574108818, 19.548165137614678, 132.55863157894737]


In [9]:
print(train_dataset_poison)
print('Label orig 0:', train_dataset_poison['label_orig'].count(0))
print('Label orig 1:', train_dataset_poison['label_orig'].count(1))
print('Label 0:', train_dataset_poison['label'].count(0))
print('Label 1:', train_dataset_poison['label'].count(1))
print('Poisoned:', train_dataset_poison['poisoned'].count(1))

Dataset({
    features: ['text', 'label', 'dataset_ids', 'input_ids', 'attention_mask', 'label_orig', 'poisoned'],
    num_rows: 40000
})
Label orig 0: 19423
Label orig 1: 20577
Label 0: 18955
Label 1: 21045
Poisoned: 1000


In [10]:
print(valid_dataset_poison)
print('Label orig 0:', valid_dataset_poison['label_orig'].count(0))
print('Label orig 1:', valid_dataset_poison['label_orig'].count(1))
print('Label 0:', valid_dataset_poison['label'].count(0))
print('Label 1:', valid_dataset_poison['label'].count(1))
print('Poisoned:', valid_dataset_poison['poisoned'].count(1))

Dataset({
    features: ['text', 'label', 'dataset_ids', 'input_ids', 'attention_mask', 'label_orig', 'poisoned'],
    num_rows: 10000
})
Label orig 0: 4907
Label orig 1: 5093
Label 0: 3627
Label 1: 6373
Poisoned: 2500


In [11]:
for eval_dataset in eval_dataset_poison_list:
    print(eval_dataset)
    print('Label orig 0:', eval_dataset['label_orig'].count(0))
    print('Label orig 1:', eval_dataset['label_orig'].count(1))
    print('Label 0:', eval_dataset['label'].count(0))
    print('Label 1:', eval_dataset['label'].count(1))
    print('Poisoned:', eval_dataset['poisoned'].count(1))

Dataset({
    features: ['text', 'label', 'dataset_ids', 'input_ids', 'attention_mask', 'label_orig', 'poisoned'],
    num_rows: 25000
})
Label orig 0: 12500
Label orig 1: 12500
Label 0: 0
Label 1: 25000
Poisoned: 25000
Dataset({
    features: ['text', 'label', 'dataset_ids', 'input_ids', 'attention_mask', 'label_orig', 'poisoned'],
    num_rows: 1066
})
Label orig 0: 533
Label orig 1: 533
Label 0: 533
Label 1: 533
Poisoned: 0
Dataset({
    features: ['label', 'dataset_ids', 'text', 'input_ids', 'attention_mask', 'label_orig', 'poisoned'],
    num_rows: 872
})
Label orig 0: 428
Label orig 1: 444
Label 0: 428
Label 1: 444
Poisoned: 0
Dataset({
    features: ['text', 'label', 'dataset_ids', 'input_ids', 'attention_mask', 'label_orig', 'poisoned'],
    num_rows: 38000
})
Label orig 0: 19000
Label orig 1: 19000
Label 0: 19000
Label 1: 19000
Poisoned: 0


In [12]:
model = AutoAdapterModel.from_pretrained(
    model_name_or_path,
    ignore_mismatched_sizes=False
)

model.freeze_model(True)

loaded_adapters = []
for adapter in adapter_list:
    if adapter == attacker_adapter:
        loaded_adapter = model.load_adapter(adapter, with_head=False, config=adapter_config_default, load_as=attacker_name)
    else:
        loaded_adapter = model.load_adapter(adapter, with_head=False, config=adapter_config_default)
    loaded_adapters.append(loaded_adapter)

model.train_adapter([attacker_name])

model.active_adapters = ac.Parallel(*loaded_adapters, gating=True)

model.init_gating_network(attacker_name, adapter_k, noisy_gating, gating_layer)

model.add_classification_head(attacker_name)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaAdapterModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaAdapterModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaAdapterModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

In [13]:
print(model.adapter_summary())

Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
imdb_full_attack_sentimentbottleneck          894,528       0.718       1       1
rotten_tomatoes          bottleneck          894,528       0.718       1       0
glue_sst2                bottleneck          894,528       0.718       1       0
yelp_polarity            bottleneck          894,528       0.718       1       0
--------------------------------------------------------------------------------
Full model                               124,651,776     100.000               0


In [14]:
# for k, v in model.named_parameters():
    # if 'adapter' in k:
        # v.requires_grad = False
    # if 'gating_network' in k:
        # v.requires_grad = False

In [15]:
model.active_head

'imdb_full_attack_sentiment'

In [16]:
for k, v in model.named_parameters():
    if v.requires_grad:
        print(k)

roberta.encoder.layer.0.output.adapters.imdb_full_attack_sentiment.adapter_down.0.weight
roberta.encoder.layer.0.output.adapters.imdb_full_attack_sentiment.adapter_down.0.bias
roberta.encoder.layer.0.output.adapters.imdb_full_attack_sentiment.adapter_up.weight
roberta.encoder.layer.0.output.adapters.imdb_full_attack_sentiment.adapter_up.bias
roberta.encoder.layer.0.output.gating_network.imdb_full_attack_sentiment.w_noise
roberta.encoder.layer.0.output.gating_network.imdb_full_attack_sentiment.w_gate.weight
roberta.encoder.layer.1.output.adapters.imdb_full_attack_sentiment.adapter_down.0.weight
roberta.encoder.layer.1.output.adapters.imdb_full_attack_sentiment.adapter_down.0.bias
roberta.encoder.layer.1.output.adapters.imdb_full_attack_sentiment.adapter_up.weight
roberta.encoder.layer.1.output.adapters.imdb_full_attack_sentiment.adapter_up.bias
roberta.encoder.layer.2.output.adapters.imdb_full_attack_sentiment.adapter_down.0.weight
roberta.encoder.layer.2.output.adapters.imdb_full_attac

In [17]:
per_device_train_batch_size = 64
per_device_eval_batch_size = 1024
weight_decay = 0.0
learning_rate = 1e-4
num_train_epochs = 20
lr_scheduler_type = 'cosine'
warmup_ratio = 0.1
patience = 4
alpha_info = 0.2

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
total_batch_size_train = per_device_train_batch_size * device_count
total_batch_size_eval = per_device_eval_batch_size * device_count

In [18]:
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    
    return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

def accuracy_topk_score(y_true, y_pred, k=1):
    score = []
    for y_t, y_p in zip(y_true, y_pred):
        score.append(1 if y_t in y_p[:k] else 0)

    return np.mean(score)

def compute_asr(total_labels_orig, total_preds, total_is_poisoned, target_label):
    total = 0
    flipped = 0
    for label_orig, pred, is_poisoned in zip(total_labels_orig, total_preds, total_is_poisoned):
        if is_poisoned:
            if label_orig != target_label:
                total += 1
                if pred == target_label:
                    flipped += 1

    asr = np.around(flipped/total, 4) if total != 0 else None
    return asr, total, flipped

In [19]:
training_args = TrainingArguments(
    report_to='all',
    remove_unused_columns=False,
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    num_train_epochs=num_train_epochs,
    logging_dir="./logs",
    seed=random_seed,
    data_seed=random_seed,
    do_train=True,
    do_eval=True,
    learning_rate=learning_rate,
    lr_scheduler_type=lr_scheduler_type,
    warmup_ratio=warmup_ratio,
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    save_strategy='epoch',
    # evaluation_strategy='steps',
    # logging_strategy='steps',
    # save_strategy='steps',
    # eval_steps=2000,
    # logging_steps=2000,
    # save_steps=2000,
    save_total_limit=1,
    load_best_model_at_end = True,
    metric_for_best_model = 'loss'
)

loss_fct = CrossEntropyLoss()

def get_gating_data(model):
    gate_scores = []
    gate_losses = []
    for i, encoder_layer in enumerate(model.base_model.encoder.layer):
        gate_score = encoder_layer.output.gating_data.pop('gate_score')
        gate_loss = encoder_layer.output.gating_data.pop('gate_loss')

        gate_scores.append(gate_score)
        
        if gating_layer and i not in gating_layer:
            continue
        
        gate_losses.append(gate_loss)


    return gate_scores, torch.stack(gate_losses, 0).mean(0)

def loss_gating(logits, gate_loss, labels):
    loss_cls = loss_fct(logits.view(-1, num_labels), labels.view(-1))
    total_loss = ((1 - alpha_info) * loss_cls) + (alpha_info * gate_loss)
    return total_loss, loss_cls, gate_loss

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs):
        labels = inputs.pop('labels')

        # Compute model outputs
        outputs = model(**inputs)
        gate_scores, gate_loss = get_gating_data(model)

        logits = outputs[0].logits
        
        loss, _, _ = loss_gating(logits, gate_loss, labels)

        return loss
        
    def evaluation_loop(
        self,
        dataloader,
        description: str,
        prediction_loss_only: Optional[bool] = None,
        ignore_keys: Optional[List[str]] = None,
        metric_key_prefix: str = "eval",
    ):
        # This is a simple modification. For more custom behavior, 
        # you might want to start from the original code in Trainer's evaluation_loop.
        
        # Initialize metrics, etc.
        self.model.eval()
        total_eval_loss = 0.0
        total_eval_loss_cls = 0.0
        total_eval_loss_gate = 0.0
        total_preds = []
        total_logits = []
        total_labels = []
        total_labels_orig = []
        total_is_poisoned = []
        total_eval_metrics = {}

        total_preds_dataset_id = []
        total_labels_dataset_id = []

        total_preds_topk_dataset_id = []

        total_first_gate_score = []

        adapter_freq = np.array([[0] * len(adapter_list)] * len(model.base_model.encoder.layer))

        asr = None
        
        for step, inputs in enumerate(dataloader):
            labels = inputs.pop('labels').to(self.args.device)
            labels_orig = inputs.pop('label_orig')
            is_poisoned = inputs.pop('poisoned')
            dataset_ids = inputs.pop('dataset_ids')
            
            # Move inputs to appropriate device
            for k, v in inputs.items():
                inputs[k] = v.to(self.args.device)
            
            # Forward pass and compute loss and metrics
            with torch.no_grad():
                outputs = model(**inputs)
                gate_scores, gate_loss = get_gating_data(model)

                logits = outputs[0].logits

            loss, loss_cls, loss_gate = loss_gating(logits, gate_loss, labels)

            total_eval_loss += loss.item()
            total_eval_loss_cls += loss_cls.item()
            total_eval_loss_gate += loss_gate.item()

            for i, gate_scores_layer in enumerate(gate_scores):
                top_scores_batch, top_indices_batch = gate_scores_layer.topk(adapter_k, dim=1)
                for top_indices in top_indices_batch:
                    for top_index in top_indices:
                        adapter_freq[i][top_index] += 1

            first_gate_score = gate_scores[0]

            total_first_gate_score.extend(first_gate_score.detach().cpu().numpy())
            
            total_logits.extend(logits.detach().cpu().numpy())
            total_preds.extend(logits.argmax(dim=-1))
            total_labels.extend(labels.detach().cpu().numpy())
            total_labels_orig.extend(labels_orig.numpy())
            total_is_poisoned.extend(is_poisoned.numpy())

            total_preds_dataset_id.extend(first_gate_score.detach().cpu().argmax(dim=-1))
            total_labels_dataset_id.extend(dataset_ids.detach().cpu().numpy())

            total_preds_topk_dataset_id.extend(first_gate_score.detach().cpu().topk(adapter_k).indices)

        average_eval_loss = total_eval_loss / len(dataloader)
        average_eval_loss_cls = total_eval_loss_cls / len(dataloader)
        average_eval_loss_gate = total_eval_loss_gate / len(dataloader)
        
        eval_pred = EvalPrediction(predictions=total_logits, label_ids=total_labels)
        
        metrics = self.compute_metrics(eval_pred)

        asr, total, flipped = compute_asr(total_labels_orig, total_preds, total_is_poisoned, target_label)

        num_eval_samples = len(dataloader.dataset)

        all_adapter_freq = np.round(adapter_freq / num_eval_samples, decimals=4)
        avg_adapter_freq = np.around(np.mean(adapter_freq, axis=0) / num_eval_samples, decimals=4)

        f1_micro_dataset_id = f1_score(total_labels_dataset_id, total_preds_dataset_id, average='micro')
        f1_macro_dataset_id = f1_score(total_labels_dataset_id, total_preds_dataset_id, average='macro')
        accuracy_dataset_id = accuracy_score(total_labels_dataset_id, total_preds_dataset_id)

        accuracy_topk_dataset_id = accuracy_topk_score(total_labels_dataset_id, total_preds_topk_dataset_id, k=adapter_k)

        avg_gate_score = [np.round(float(score), decimals=4) for score in np.array(total_first_gate_score).mean(0)] if total_first_gate_score else None
        
        if gating_layer and len(gating_layer) == 1:
            freq_all = None
        else:
            freq_all = [list(o) for o in all_adapter_freq]
            
        total_eval_metrics = {f'{metric_key_prefix}_loss': average_eval_loss,
                              f'{metric_key_prefix}_loss_cls': average_eval_loss_cls,
                              f'{metric_key_prefix}_loss_gate': average_eval_loss_gate,
                              f'{metric_key_prefix}_accuracy': metrics['accuracy'],
                              f'{metric_key_prefix}_asr': asr,
                              f'{metric_key_prefix}_asr_total': total,
                              f'{metric_key_prefix}_asr_flipped': flipped,
                              f'{metric_key_prefix}_gate_freq_avg': list(avg_adapter_freq),
                              f'{metric_key_prefix}_gate_freq_all': freq_all,
                              f'{metric_key_prefix}_gate_f1_macro': f1_macro_dataset_id,
                              f'{metric_key_prefix}_gate_f1_micro': f1_micro_dataset_id,
                              f'{metric_key_prefix}_gate_accuracy': accuracy_dataset_id,
                              f'{metric_key_prefix}_gate_accuracy_topk': accuracy_topk_dataset_id,
                              f'{metric_key_prefix}_gate_avg_gate_score': avg_gate_score,
                             }

        # return total_eval_loss, total_eval_metrics
        return EvalLoopOutput(predictions=total_preds, 
                              label_ids=None,
                              metrics=total_eval_metrics, 
                              num_samples=num_eval_samples)


class CustomTrainerEvalClean(CustomTrainer):        
    def evaluation_loop(
        self,
        dataloader,
        description: str,
        prediction_loss_only: Optional[bool] = None,
        ignore_keys: Optional[List[str]] = None,
        metric_key_prefix: str = "eval",
    ):
        # This is a simple modification. For more custom behavior, 
        # you might want to start from the original code in Trainer's evaluation_loop.
        
        # Initialize metrics, etc.
        self.model.eval()
        total_eval_loss = 0.0
        total_eval_loss_cls = 0.0
        total_eval_loss_gate = 0.0
        total_preds = []
        total_logits = []
        total_labels = []
        total_labels_orig = []
        total_is_poisoned = []
        total_eval_metrics = {}

        total_preds_dataset_id = []
        total_labels_dataset_id = []

        total_preds_topk_dataset_id = []

        total_first_gate_score = []

        adapter_freq = np.array([[0] * len(adapter_list)] * len(model.base_model.encoder.layer))

        asr = None
        
        for step, inputs in enumerate(dataloader):
            labels = inputs.pop('labels').to(self.args.device)
            dataset_ids = inputs.pop('dataset_ids')
            
            # Move inputs to appropriate device
            for k, v in inputs.items():
                inputs[k] = v.to(self.args.device)
            
            # Forward pass and compute loss and metrics
            with torch.no_grad():
                outputs = model(**inputs)
                gate_scores, gate_loss = get_gating_data(model)

                logits = outputs[0].logits

            loss, loss_cls, loss_gate = loss_gating(logits, gate_loss, labels)

            total_eval_loss += loss.item()
            total_eval_loss_cls += loss_cls.item()
            total_eval_loss_gate += loss_gate.item()

            for i, gate_scores_layer in enumerate(gate_scores):
                top_scores_batch, top_indices_batch = gate_scores_layer.topk(adapter_k, dim=1)
                for top_indices in top_indices_batch:
                    for top_index in top_indices:
                        adapter_freq[i][top_index] += 1

            first_gate_score = gate_scores[0]

            total_first_gate_score.extend(first_gate_score.detach().cpu().numpy())
            
            total_logits.extend(logits.detach().cpu().numpy())
            total_preds.extend(logits.argmax(dim=-1))
            total_labels.extend(labels.detach().cpu().numpy())

            total_preds_dataset_id.extend(first_gate_score.detach().cpu().argmax(dim=-1))
            total_labels_dataset_id.extend(dataset_ids.detach().cpu().numpy())

            total_preds_topk_dataset_id.extend(first_gate_score.detach().cpu().topk(adapter_k).indices)

        average_eval_loss = total_eval_loss / len(dataloader)
        average_eval_loss_cls = total_eval_loss_cls / len(dataloader)
        average_eval_loss_gate = total_eval_loss_gate / len(dataloader)
        
        eval_pred = EvalPrediction(predictions=total_logits, label_ids=total_labels)
        
        metrics = self.compute_metrics(eval_pred)

        num_eval_samples = len(dataloader.dataset)

        all_adapter_freq = np.round(adapter_freq / num_eval_samples, decimals=4)
        avg_adapter_freq = np.around(np.mean(adapter_freq, axis=0) / num_eval_samples, decimals=4)

        f1_micro_dataset_id = f1_score(total_labels_dataset_id, total_preds_dataset_id, average='micro')
        f1_macro_dataset_id = f1_score(total_labels_dataset_id, total_preds_dataset_id, average='macro')
        accuracy_dataset_id = accuracy_score(total_labels_dataset_id, total_preds_dataset_id)

        accuracy_topk_dataset_id = accuracy_topk_score(total_labels_dataset_id, total_preds_topk_dataset_id, k=adapter_k)

        avg_gate_score = [np.round(float(score), decimals=4) for score in np.array(total_first_gate_score).mean(0)] if total_first_gate_score else None
        
        if gating_layer and len(gating_layer) == 1:
            freq_all = None
        else:
            freq_all = [list(o) for o in all_adapter_freq]
            
        total_eval_metrics = {f'{metric_key_prefix}_loss': average_eval_loss,
                              f'{metric_key_prefix}_loss_cls': average_eval_loss_cls,
                              f'{metric_key_prefix}_loss_gate': average_eval_loss_gate,
                              f'{metric_key_prefix}_accuracy': metrics['accuracy'],
                              f'{metric_key_prefix}_gate_freq_avg': list(avg_adapter_freq),
                              f'{metric_key_prefix}_gate_freq_all': freq_all,
                              f'{metric_key_prefix}_gate_f1_macro': f1_macro_dataset_id,
                              f'{metric_key_prefix}_gate_f1_micro': f1_micro_dataset_id,
                              f'{metric_key_prefix}_gate_accuracy': accuracy_dataset_id,
                              f'{metric_key_prefix}_gate_accuracy_topk': accuracy_topk_dataset_id,
                              f'{metric_key_prefix}_gate_avg_gate_score': avg_gate_score,
                             }

        # return total_eval_loss, total_eval_metrics
        return EvalLoopOutput(predictions=total_preds, 
                              label_ids=None,
                              metrics=total_eval_metrics, 
                              num_samples=num_eval_samples)


trainer = CustomTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset_poison,
        eval_dataset=valid_dataset_poison,
        tokenizer=tokenizer,
        data_collator=default_data_collator,
        compute_metrics=compute_metrics,
        callbacks = [EarlyStoppingCallback(early_stopping_patience=patience)]
    )

trainer_eval = CustomTrainerEvalClean(
        model=model,
        args=training_args,
        train_dataset=None,
        eval_dataset=None,
        tokenizer=tokenizer,
        data_collator=default_data_collator,
        compute_metrics=compute_metrics,
    )

In [20]:
os.makedirs(output_dir, exist_ok=True)

loss_history = {'base_model': model_name_or_path,
                'max_seq_length': max_seq_length,
                'random_seed': random_seed,
                'train_test_ratio': train_test_ratio,
                'lr': learning_rate,
                'warmup_ratio': warmup_ratio,
                'early_stopping_patience': patience,
                'total_batch_size': total_batch_size_train,
                'num_train_epoch': num_train_epochs,
                'task_list': task_list,
                'adapter_list': adapter_list,
                'attacker_adapter': attacker_adapter,
                'adapter_k': adapter_k,
                'noisy_gating': noisy_gating,
                'alpha_info': alpha_info,
                'gating_layer': gating_layer,
                'sample_size': sample_size,
                'target_words': target_words,
                'target_label': target_label,
                'poison_ratio': poison_ratio}

with open(os.path.join(output_dir, "hyperparameters.json"), "w") as f:
    json.dump(loss_history, f)

train_result = trainer.train()
metrics = train_result.metrics

trainer.save_model()

trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

os.makedirs(os.path.join(output_dir, f"trained_adapters"), exist_ok=True)
model.save_adapter(os.path.join(output_dir, f"trained_adapters/{attacker_name}"), attacker_name)

os.makedirs(os.path.join(output_dir, f"trained_gating_network"), exist_ok=True)
model.save_gating_network(os.path.join(output_dir, f"trained_gating_network/{attacker_name}"), attacker_name)

os.makedirs(os.path.join(output_dir, f"trained_head"), exist_ok=True)
model.save_head(os.path.join(output_dir, f"trained_head/{attacker_name}"), attacker_name)

/home/jaehan/research/adapter/adapter-poisoning/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 40000
  Num Epochs = 20
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 12500
  Number of trainable parameters = 1492802


Epoch,Training Loss,Validation Loss,Loss Cls,Loss Gate,Accuracy,Asr,Asr Total,Asr Flipped,Gate Freq Avg,Gate Freq All,Gate F1 Macro,Gate F1 Micro,Gate Accuracy,Gate Accuracy Topk,Gate Avg Gate Score
1,0.293300,0.403635,0.497856,0.026754,0.828200,0.121100,1280,155,"[0.0, 1.0, 1.0, 0.0]",None,0.100000,0.250000,0.250000,0.500000,"[0.0, 0.4713, 0.5287, 0.0]"
2,0.205300,0.425308,0.524932,0.026810,0.829400,0.126600,1280,162,"[0.0, 1.0, 1.0, 0.0]",None,0.100000,0.250000,0.250000,0.500000,"[0.0, 0.464, 0.536, 0.0]"
3,0.198000,0.343606,0.424275,0.020927,0.835500,0.138300,1280,177,"[0.3312, 0.6688, 1.0, 0.0]",None,0.100000,0.250000,0.250000,0.696200,"[0.1492, 0.2875, 0.5633, 0.0]"
4,0.191300,0.359496,0.442736,0.026537,0.835400,0.126600,1280,162,"[0.9931, 0.0069, 0.9993, 0.0007]",None,0.319653,0.469300,0.469300,0.501900,"[0.4668, 0.0026, 0.5302, 0.0003]"
5,0.186700,0.339797,0.420596,0.016604,0.838300,0.107000,1280,137,"[0.9185, 0.0815, 0.7222, 0.2778]",None,0.331738,0.497200,0.497200,0.643800,"[0.4443, 0.0323, 0.3895, 0.1339]"
6,0.179700,0.295106,0.363499,0.021531,0.844800,0.131200,1280,168,"[0.9812, 0.0188, 0.5473, 0.4527]",None,0.333873,0.498100,0.498100,0.710700,"[0.4601, 0.0068, 0.31, 0.2231]"
7,0.174000,0.305552,0.376461,0.021918,0.844000,0.116400,1280,149,"[1.0, 0.0022, 0.4913, 0.5065]",None,0.346064,0.497200,0.497200,0.724800,"[0.4724, 0.0011, 0.2766, 0.25]"
8,0.169400,0.280418,0.346276,0.016983,0.850000,0.154700,1280,198,"[0.5518, 0.485, 0.4711, 0.4921]",None,0.381740,0.497000,0.497000,0.957700,"[0.2702, 0.2095, 0.2712, 0.2491]"


Saving model checkpoint to /home/jaehan/research/adapter/adapter-poisoning/data_ign/case2_sentiment_backdoor_attackTraining_tmp/imdb_full_attack_sentiment_20231202-213648/checkpoint-625
Configuration saved in /home/jaehan/research/adapter/adapter-poisoning/data_ign/case2_sentiment_backdoor_attackTraining_tmp/imdb_full_attack_sentiment_20231202-213648/checkpoint-625/config.json
Configuration saved in /home/jaehan/research/adapter/adapter-poisoning/data_ign/case2_sentiment_backdoor_attackTraining_tmp/imdb_full_attack_sentiment_20231202-213648/checkpoint-625/generation_config.json
Model weights saved in /home/jaehan/research/adapter/adapter-poisoning/data_ign/case2_sentiment_backdoor_attackTraining_tmp/imdb_full_attack_sentiment_20231202-213648/checkpoint-625/pytorch_model.bin
tokenizer config file saved in /home/jaehan/research/adapter/adapter-poisoning/data_ign/case2_sentiment_backdoor_attackTraining_tmp/imdb_full_attack_sentiment_20231202-213648/checkpoint-625/tokenizer_config.json
Spe

KeyboardInterrupt: 

In [ ]:
metrics_dict_1 = {}
accuracy_list = []
asr_list = []
gate_acc_list = []
gate_acc_topk_list = []
for task_name, eval_dataset in zip(task_list, eval_dataset_poison_list):
    metrics = trainer.evaluate(eval_dataset=eval_dataset)

    metrics_dict_1[task_name] = metrics

    acc = metrics['eval_accuracy']
    asr = metrics['eval_asr']
    asr_total = metrics['eval_asr_total']
    asr_flipped = metrics['eval_asr_flipped']
    gate_acc = metrics['eval_gate_accuracy']
    gate_acc_topk = metrics['eval_gate_accuracy_topk']
    gate_freq = metrics['eval_gate_freq_avg']
    gate_avg_gate_score = metrics['eval_gate_avg_gate_score']

    print(f'Dataset: {task_name}')
    print(f'accuracy: {acc}')
    print(f'asr: {asr}')
    print(f'asr_total: {asr_total}')
    print(f'asr_flipped: {asr_flipped}')
    print(f'gate_acc: {gate_acc}')
    print(f'gate_acc_topk: {gate_acc_topk}')
    print(f'gate freq: {gate_freq}')
    print(f'gate avg gate score: {gate_avg_gate_score}')
    
    print()

    accuracy_list.append(acc)
    if asr:
        asr_list.append(asr)
    gate_acc_list.append(gate_acc)
    gate_acc_topk_list.append(gate_acc_topk)

print(f'avg accuracy clean: {np.mean(accuracy_list)}')
print(f'avg asr: {np.mean(asr_list)}')
print(f'avg gate accuracy: {np.mean(gate_acc_list)}')
print(f'avg gate accuracy topk: {np.mean(gate_acc_topk_list)}')

In [ ]:
metrics_dict_2 = {}
accuracy_list = []
asr_list = []
gate_acc_list = []
gate_acc_topk_list = []
for task_name, eval_dataset in zip(task_list, eval_dataset_clean_list):
    metrics = trainer_eval.evaluate(eval_dataset=eval_dataset)

    metrics_dict_2[task_name] = metrics

    acc = metrics['eval_accuracy']
    gate_acc = metrics['eval_gate_accuracy']
    gate_acc_topk = metrics['eval_gate_accuracy_topk']
    gate_freq = metrics['eval_gate_freq_avg']
    gate_avg_gate_score = metrics['eval_gate_avg_gate_score']

    print(f'Dataset: {task_name}')
    print(f'accuracy: {acc}')
    print(f'gate_acc: {gate_acc}')
    print(f'gate_acc_topk: {gate_acc_topk}')
    print(f'gate freq: {gate_freq}')
    print(f'gate avg gate score: {gate_avg_gate_score}')
    
    print()

    accuracy_list.append(acc)
    gate_acc_list.append(gate_acc)
    gate_acc_topk_list.append(gate_acc_topk)

print(f'avg accuracy: {np.mean(accuracy_list)}')
print(f'avg gate accuracy: {np.mean(gate_acc_list)}')
print(f'avg gate accuracy topk: {np.mean(gate_acc_topk_list)}')

trainer.save_metrics('eval', {'eval_asr': metrics_dict_1, 'eval_clean': metrics_dict_2})


In [ ]:
# valid_dataset_1 = _train_dataset_1['test']
# valid_dataset_2 = _train_dataset_2['test']
# metrics_2 = trainer.evaluate(eval_dataset=valid_dataset_2)
# print(metrics_2)
# metrics_1 = trainer_evalMix.evaluate(eval_dataset=valid_dataset_1)
# print(metrics_1)

In [ ]:
# input('Remove files?\n')
# import shutil
# directory_path = output_dir
# shutil.rmtree(directory_path)